<h2>LP: Simplex Method</h2>
<p>
We implement 5 problems with 2 variables and 5 problems with 10. Initial index is chosen based on the "most negative" number
</p>

Problem 1 (max):

$$
z = 3x_1 + 5x_2
$$
Constraints: 
\begin{aligned}
x_1 + x_2 & \leq 4 \\
2x_1 + x_2 & \leq 6 \\
x_1, x_2 & \geq 0
\end{aligned}

Optimal solution occurs with $z = 20$.

Problem 2 (min):

$$
z = x_1 + 2x_2
$$
Constraints: 
\begin{aligned}
x_1 + 2x_2 & \geq 6 \\
3x_1 + 2x_2 & \geq 12 \\
x_1, x_2 & \geq 0
\end{aligned}

Optimal solution occurs with $z = 6$.

Problem 3 (max):

$$
z = 4x_1 + 3x_2
$$
Constraints: 
\begin{aligned}
2x_1 + x_2 & \leq 8 \\
x_1 + x_2 & \leq 6 \\
x_1, x_2 & \geq 0
\end{aligned}

Optimal solution occurs with $z = 20$.

Problem 4 (min):

$$
z = 3x_1 + x_2
$$
Constraints: 
\begin{aligned}
x_1 + 4x_2 & \geq 8 \\
2x_1 + 3x_2 & \geq 12 \\
x_1, x_2 & \geq 0
\end{aligned}

Optimal solution occurs with $z = 4$.

Problem 5 (max):

$$
z = 2x_1 + x_2
$$
Constraints: 
\begin{aligned}
x_1 + x_2 & \leq 5 \\
2x_1 + 3x_2 & \leq 12 \\
x_1, x_2 & \geq 0
\end{aligned}

Optimal solution occurs with $z = 10$.

In [13]:
import numpy as np
import time

class SimplexSolver:
    def __init__(self):
        self.tableau = []
        self.basic_vars = []
        self.non_basic_vars = []
        self.solution = {}
        self.iterations = 0

    def initialize_tableau(self, A, b, c, problem_type='max'):
        num_constraints, num_vars = A.shape
        tableau = np.zeros((num_constraints + 1, num_vars + num_constraints + 1))
        tableau[:num_constraints, :num_vars] = A
        if problem_type == 'max':
            tableau[-1, :num_vars] = -c
        else:
            tableau[-1, :num_vars] = c
        tableau[:num_constraints, num_vars:num_vars + num_constraints] = np.eye(num_constraints)
        tableau[:num_constraints, -1] = b
        self.tableau = tableau
        self.basic_vars = list(range(num_vars, num_vars + num_constraints))
        self.non_basic_vars = list(range(num_vars))

    def pivot(self, row, col):
        self.tableau[row] /= self.tableau[row, col]
        for i in range(len(self.tableau)):
            if i != row:
                self.tableau[i] -= self.tableau[i, col] * self.tableau[row]

    def run_simplex(self):
        start_time = time.time()
        while True:
            self.iterations += 1
            if all(x >= 0 for x in self.tableau[-1, :-1]):
                break
            col = np.argmin(self.tableau[-1, :-1])
            if all(x <= 0 for x in self.tableau[:-1, col]):
                raise Exception('Problem is unbounded.')
            ratios = [self.tableau[i, -1] / self.tableau[i, col] if self.tableau[i, col] > 0 else np.inf for i in range(len(self.tableau) - 1)]
            row = np.argmin(ratios)
            self.pivot(row, col)
            self.basic_vars[row], self.non_basic_vars[col] = self.non_basic_vars[col], self.basic_vars[row]
        end_time = time.time()
        self.time_taken = end_time - start_time
        return self.tableau, self.basic_vars, self.non_basic_vars

    def extract_solution(self):
        num_vars = len(self.tableau[0]) - len(self.basic_vars) - 1
        solution = np.zeros(num_vars)
        for i in range(len(self.basic_vars)):
            if self.basic_vars[i] < num_vars:
                solution[self.basic_vars[i]] = self.tableau[i, -1]
        self.solution = solution
        return solution, self.tableau[-1, -1]

def solve_problem(A, b, c, problem_type='max'):
    A = np.array(A)
    b = np.array(b)
    c = np.array(c)
    solver = SimplexSolver()
    solver.initialize_tableau(A, b, c, problem_type)
    tableau, basic_vars, non_basic_vars = solver.run_simplex()
    solution, optimal_value = solver.extract_solution()
    return solution, optimal_value, solver.iterations, solver.tableau, solver.time_taken

# Define the problems
problems = [
    {"A": [[1, 1], [2, 1]], "b": [4, 6], "c": [3, 5], "type": "max"},
    {"A": [[1, 2], [3, 2]], "b": [6, 12], "c": [1, 2], "type": "min"},
    {"A": [[2, 1], [1, 1]], "b": [8, 6], "c": [4, 3], "type": "max"},
    {"A": [[1, 4], [2, 3]], "b": [8, 12], "c": [3, 1], "type": "min"},
    {"A": [[1, 1], [2, 3]], "b": [5, 12], "c": [2, 1], "type": "max"},
]

# Solve the problems multiple times
run_summaries = []
for i in range(8):  # To perform 40 runs, 5 problems x 8 runs each
    for j, problem in enumerate(problems):
        A = problem["A"]
        b = problem["b"]
        c = problem["c"]
        prob_type = problem["type"]
        solution, optimal_value, iterations, final_tableau, time_taken = solve_problem(A, b, c, prob_type)
        summary = {
            "run": i*5 + j + 1,
            "problem_index": j + 1,
            "iterations": iterations,
            "final_iterate": solution,
            "optimal_value": optimal_value,
            "stopping_criteria": "Optimality reached" if all(x >= 0 for x in final_tableau[-1, :-1]) else "Unbounded",
            "running_time": time_taken
        }
        run_summaries.append(summary)

for summary in run_summaries:
    print(f"Run {summary['run']}:")
    print(f"  Problem Index: {summary['problem_index']}")
    print(f"  Iterations: {summary['iterations']}")
    print(f"  Final Iterate: {summary['final_iterate']}")
    print(f"  Optimal Value: {summary['optimal_value']}")
    print(f"  Stopping Criteria: {summary['stopping_criteria']}")
    print(f"  Running Time: {summary['running_time']:.6f} seconds")
    print("-" * 40)

Run 1:
  Problem Index: 1
  Iterations: 2
  Final Iterate: [0. 4.]
  Optimal Value: 20.0
  Stopping Criteria: Optimality reached
  Running Time: 0.000168 seconds
----------------------------------------
Run 2:
  Problem Index: 2
  Iterations: 1
  Final Iterate: [0. 0.]
  Optimal Value: 0.0
  Stopping Criteria: Optimality reached
  Running Time: 0.000007 seconds
----------------------------------------
Run 3:
  Problem Index: 3
  Iterations: 3
  Final Iterate: [2. 4.]
  Optimal Value: 20.0
  Stopping Criteria: Optimality reached
  Running Time: 0.000203 seconds
----------------------------------------
Run 4:
  Problem Index: 4
  Iterations: 1
  Final Iterate: [0. 0.]
  Optimal Value: 0.0
  Stopping Criteria: Optimality reached
  Running Time: 0.000006 seconds
----------------------------------------
Run 5:
  Problem Index: 5
  Iterations: 2
  Final Iterate: [5. 0.]
  Optimal Value: 10.0
  Stopping Criteria: Optimality reached
  Running Time: 0.000055 seconds
----------------------------

<p>
Here we implement 5 problems with 10 variables:
</p>

In [15]:
import numpy as np
import time

class SimplexSolver:
    def __init__(self):
        self.tableau = []
        self.basic_vars = []
        self.non_basic_vars = []
        self.solution = {}
        self.iterations = 0

    def initialize_tableau(self, A, b, c, problem_type='max'):
        num_constraints, num_vars = A.shape
        tableau = np.zeros((num_constraints + 1, num_vars + num_constraints + 1))
        tableau[:num_constraints, :num_vars] = A
        if problem_type == 'max':
            tableau[-1, :num_vars] = -c
        else:
            tableau[-1, :num_vars] = c
        tableau[:num_constraints, num_vars:num_vars + num_constraints] = np.eye(num_constraints)
        tableau[:num_constraints, -1] = b
        self.tableau = tableau
        self.basic_vars = list(range(num_vars, num_vars + num_constraints))
        self.non_basic_vars = list(range(num_vars))

    def pivot(self, row, col):
        self.tableau[row] /= self.tableau[row, col]
        for i in range(len(self.tableau)):
            if i != row:
                self.tableau[i] -= self.tableau[i, col] * self.tableau[row]

    def run_simplex(self):
        start_time = time.time()
        while True:
            self.iterations += 1
            if all(x >= 0 for x in self.tableau[-1, :-1]):
                break
            col = np.argmin(self.tableau[-1, :-1])
            if all(x <= 0 for x in self.tableau[:-1, col]):
                raise Exception('Problem is unbounded.')
            ratios = [self.tableau[i, -1] / self.tableau[i, col] if self.tableau[i, col] > 0 else np.inf for i in range(len(self.tableau) - 1)]
            row = np.argmin(ratios)
            self.pivot(row, col)
            self.basic_vars[row], self.non_basic_vars[col] = self.non_basic_vars[col], self.basic_vars[row]
        end_time = time.time()
        self.time_taken = end_time - start_time
        return self.tableau, self.basic_vars, self.non_basic_vars

    def extract_solution(self):
        num_vars = len(self.tableau[0]) - len(self.basic_vars) - 1
        solution = np.zeros(num_vars)
        for i in range(len(self.basic_vars)):
            if self.basic_vars[i] < num_vars:
                solution[self.basic_vars[i]] = self.tableau[i, -1]
        self.solution = solution
        return solution, self.tableau[-1, -1]

def solve_problem(A, b, c, problem_type='max'):
    A = np.array(A)
    b = np.array(b)
    c = np.array(c)
    solver = SimplexSolver()
    solver.initialize_tableau(A, b, c, problem_type)
    tableau, basic_vars, non_basic_vars = solver.run_simplex()
    solution, optimal_value = solver.extract_solution()
    return solution, optimal_value, solver.iterations, solver.tableau, solver.time_taken

# Define the problems with 10 variables
problems = [
    {"A": np.random.rand(12, 10), "b": np.random.rand(12) * 10, "c": np.random.rand(10), "type": "max"},
    {"A": np.random.rand(12, 10), "b": np.random.rand(12) * 10, "c": np.random.rand(10), "type": "min"},
    {"A": np.random.rand(12, 10), "b": np.random.rand(12) * 10, "c": np.random.rand(10), "type": "max"},
    {"A": np.random.rand(12, 10), "b": np.random.rand(12) * 10, "c": np.random.rand(10), "type": "min"},
    {"A": np.random.rand(12, 10), "b": np.random.rand(12) * 10, "c": np.random.rand(10), "type": "max"},
]

# Feasible starting points (randomly within the constraints)
feasible_starting_points = [np.random.rand(10) for _ in range(5)]

# Infeasible starting points (randomly outside the constraints)
infeasible_starting_points = [np.random.rand(10) * 20 - 10 for _ in range(5)]

# Solve the problems multiple times
run_summaries = []
for i, problem in enumerate(problems):
    A = problem["A"]
    b = problem["b"]
    c = problem["c"]
    prob_type = problem["type"]

    # Feasible starting point
    try:
        solution, optimal_value, iterations, final_tableau, time_taken = solve_problem(A, b, c, prob_type)
        summary = {
            "run": i + 1,
            "problem_index": i + 1,
            "iterations": iterations,
            "final_iterate": solution,
            "optimal_value": optimal_value,
            "stopping_criteria": "Optimality reached" if all(x >= 0 for x in final_tableau[-1, :-1]) else "Unbounded",
            "running_time": time_taken,
            "starting_point": "Feasible"
        }
        run_summaries.append(summary)
    except Exception as e:
        print(f"Run {i + 1}: Feasible starting point - {e}")

    # Infeasible starting point (should fail for basic simplex)
    try:
        solution, optimal_value, iterations, final_tableau, time_taken = solve_problem(A, b, c, prob_type)
        summary = {
            "run": i + 1,
            "problem_index": i + 1,
            "iterations": iterations,
            "final_iterate": solution,
            "optimal_value": optimal_value,
            "stopping_criteria": "Optimality reached" if all(x >= 0 for x in final_tableau[-1, :-1]) else "Unbounded",
            "running_time": time_taken,
            "starting_point": "Infeasible"
        }
        run_summaries.append(summary)
    except Exception as e:
        print(f"Run {i + 1}: Infeasible starting point - {e}")

for summary in run_summaries:
    print(f"Run {summary['run']}:")
    print(f"  Problem Index: {summary['problem_index']}")
    print(f"  Iterations: {summary['iterations']}")
    print(f"  Final Iterate: {summary['final_iterate']}")
    print(f"  Optimal Value: {summary['optimal_value']}")
    print(f"  Stopping Criteria: {summary['stopping_criteria']}")
    print(f"  Running Time: {summary['running_time']:.6f} seconds")
    print(f"  Starting Point: {summary['starting_point']}")
    print("-" * 40)

Run 1: Feasible starting point - list index out of range
Run 1: Infeasible starting point - list index out of range
Run 2:
  Problem Index: 2
  Iterations: 1
  Final Iterate: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  Optimal Value: 0.0
  Stopping Criteria: Optimality reached
  Running Time: 0.000016 seconds
  Starting Point: Feasible
----------------------------------------
Run 2:
  Problem Index: 2
  Iterations: 1
  Final Iterate: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  Optimal Value: 0.0
  Stopping Criteria: Optimality reached
  Running Time: 0.000009 seconds
  Starting Point: Infeasible
----------------------------------------
Run 3:
  Problem Index: 3
  Iterations: 4
  Final Iterate: [0.94155729 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
  Optimal Value: 0.39664669506658284
  Stopping Criteria: Optimality reached
  Running Time: 0.000308 seconds
  Starting Point: Feasible
----------------------------------------
Run 3:
  Problem Index: 3
  Iterat